In [2]:
from gurobipy import *
import pandas as pd
import numpy as np
import random

Load dataset

In [3]:
movie_data = pd.read_csv('./MSP_dataset.csv')
U = movie_data['userId'].to_list()
Ir = movie_data['movieId'].to_list()
R = movie_data['retailerId'].to_list()

Constants 

In [4]:
K = 100000
m = len(set(U))
k = 9
w = 4

Declare and Initialise m1 and m2

In [5]:
m1 = Model('LUBP1')
m2 = Model('LUBP2')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-24


Data matrices x, y and u

In [6]:
x_data = list(zip(Ir, U))
y_data = list(zip(R, U))

x = np.zeros((len(Ir)+1,len(U)+1))
y = np.zeros((len(R)+1,len(U)+1))

for (i, j) in x_data:
    x[i-1, j-1] = 1

for (r, j) in y_data:
    x[r-1, j-1] = 1
    
u = np.random.randint(2, size=(len(Ir)+1))

Decision variables and Constraints for m1

In [ ]:

x_1 = m1.addMVar((len(U)+1, len(U)+1), vtype=GRB.BINARY, name="x_m1") #c3
m1.update

for i in Ir:
    m1.addConstr(quicksum(x_matrix[i-1,j-1] * x_1[i-1,j-1] for j in U) == k, 'c1%s' % str([i,j]))

Decision variables and Constraints for m2

In [ ]:
x_2 = m2.addVars(len(U)+1, len(U)+1, vtype=GRB.BINARY, name="x_m2") #c9
y_2 = m2.addVars(len(U)+1, len(U)+1, vtype=GRB.BINARY, name="y_m2") #c10
m2.update

for i in Ir:
    m2.addConstr(quicksum(x_matrix[i-1,j-1] * x_2[i-1,j-1] for j in U) == k, 'c4%s' % str([i,j]))

for i in Ir: 
    m2.addConstr(quicksum(x_matrix[i-1,j-1] * x_2[i-1,j-1] for j in U) <= quicksum(K * y_matrix[r-1, j-1] for r in R for j in U), 'c6%s' % str([i,j]))

for i in Ir: 
    m2.addConstr(quicksum(x_matrix[i-1,j-1] * x_2[i-1,j-1] for j in U) <= 1 - K + quicksum(K * y_matrix[r-1, j-1] for r in R for j in U), 'c7%s' % str([i,j]))

for r in R: 
    m2.addConstr(quicksum(y_matrix[r-1,j-1] * y_2[r-1,j-1] for j in U) >= w, 'c8%s' % str([i,j]))

Save models for inspection

In [189]:
m1.write('LUBP1.rlp')
m2.write('LUBP2.rlp')

Initial Objective Functions for m1 and m2

In [191]:
l = np.array([0.5] * 502)
a = np.array([0.5] * 502)
#Constant pr declaration - based on normal distribution over RetailerId with mean = 7.5 and standard deviation 4.82
p = np.array([0.8, 0.6, 0.4, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.4, 0.6, 0.6, 0.6, 0.4, 0.6, 0.6, 0.6, 0.8, 0.6])
c = np.multiply(p, m*k)

In [ ]:
# generator_m1 = np.matmul(u_matrix, x_1)
# generator_m2 = np.matmul(u_matrix, x_2)

init_obj_func_m1 = m1.setObjective(quicksum(u_matrix[i-1,j-1]*x_1[i-1,j-1] for i in Ir for j in U) - quicksum(l[r-1]*a[r-1]*c[r-1] for r in R), GRB.MAXIMIZE) # l[r] = 0 initially.
m1.write("LUBP1.rlp")
init_obj_func_m2 = m2.setObjective(quicksum(u_matrix[i-1,j-1]*x_1[i-1,j-1] for i in Ir for j in U) - quicksum(l[r-1]*a[r-1]*c[r-1] for r in R), GRB.MAXIMIZE) # l[r] = 0 initially.
# init_obj_func_m1 = m1.setMObjective(None, u_matrix@x_2, GRB.MAXIMIZE)
# init_obj_func_m2 = m2.setMObjective(None, np.matmul(u_matrix, x_2), 0.0, sense=GRB.MAXIMIZE)

Run optimizer for m1 and m2

In [ ]:
m1.optimize()
m2.optimize()

Get objective values for m1 and m2

In [ ]:
init_obj_m1 = m1.getObjective()
print(init_obj_m1.getValue())

init_obj_m2 = m2.getObjective()
print(init_obj_m2.getValue())

In [ ]:
def subGradientOptimisation(init_feasable_value, x):
    Z_lb = init_feasable_value
    Z_ub = 0 # variable declaration
    pi = 0.1
    l =[0] * (len(U)-1)
    # Z_lb = initial feasable soln from M1(M2)
    # Z_ub = feasable soln for M1(M2) after using all multipliers
    G = sum([x[i-1][j-1] for i in Ir for j in U])
    T = pi*(Z_ub - Z_lb) / sum(G[r-1]*G[r-1] for r in R)
    #Z_lb update for new feasable solution from M1(M2)
    for r in R:
        l[r-1] = max(0, l[r-1]+T*G[r-1])
    return l